In [ ]:
import torch
import pandas as pd
from dataset_annotation_preparation import prepare_annotation
from torch.utils.data import DataLoader

from training_evaluation import (
    test_model,
)

from utils import image_utils, model_utils

from dataset_preloader import VeriImageDatasetTest

from veri_models import ObjectVeriSiamese, ModelEnsembler


from torch.utils.data import DataLoader


# Import project configuration
from config import ANNOTATIONS_PATH, RAW_ANNOTATIONS_PATH, DATA_PATH, MODELS_PATH

Model Simple

In [ ]:
model = ObjectVeriSiamese(backbone="mobilenet_v3_small")
model.load_state_dict(torch.load(MODELS_PATH / "model_2.pth"))

Model Ensembler

In [ ]:
model1 = ObjectVeriSiamese()
model1.load_state_dict(torch.load(MODELS_PATH / "model_1.pth"))

model2 = ObjectVeriSiamese(backbone="mobilenet_v3_small")
model2.load_state_dict(torch.load(MODELS_PATH / "model_2.pth"))


model = ModelEnsembler(
    models=[
        model1,
        model2,
    ]
)

Model Save .pt File

In [ ]:
PATH = MODELS_PATH / "model_veri_eval.pt"
torch.save(model, PATH)
model = torch.load(PATH, weights_only=False)
model.eval()

In [ ]:
# %% -------------------------
# Define parameters
# -------------------------
train_ratio = 0.5
n_error = 2
n_augmentation = 5

load = True

transform_type_val = "test"

In [ ]:
raw_annotation_sharks = RAW_ANNOTATIONS_PATH / "raw_annotations_sharks.csv"
preprocessed_annotation_sharks = (
    ANNOTATIONS_PATH / "preprocessed_annotations_sharks.csv"
)

raw_annotation_birds = RAW_ANNOTATIONS_PATH / "raw_annotations_birds.csv"
preprocessed_annotation_birds = ANNOTATIONS_PATH / "preprocessed_annotations_birds.csv"

images_dir = DATA_PATH / "animals"

In [ ]:
df_test = prepare_annotation(
    raw_annotation_path=raw_annotation_birds,
    images_dir=images_dir,
    preprocessed_annotation_path=preprocessed_annotation_birds,
    train_ratio=0,
    n_augmentation=n_augmentation,
    n_error=n_error,
)

In [ ]:
transform_test = image_utils.transform_fc(transform_type_val)

test_data = VeriImageDatasetTest(
    annotations_file=preprocessed_annotation_birds,
    transform=transform_test,
)

In [ ]:
test_data.visualize_images(n=1)

In [ ]:
test_dataloader = DataLoader(
    test_data,
    batch_size=64,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

In [ ]:
df_annotation, results, quartiles_results, FRR_results = test_model(
    model, test_dataloader, print_results=True, device="cpu"
)

In [ ]:
y_true, y_scores = (
    df_annotation["label"].to_numpy(),
    df_annotation["distance"].to_numpy(),
)

In [ ]:
model_utils.plot_roc_curve(y_true, y_scores)

In [ ]:
model_utils.plot_score_distributions(y_true, y_scores, threshold=0.75)

In [ ]:
df_shuffled = df_annotation.sample(frac=1).reset_index(drop=True)
df_error = df_shuffled[
    (1 == df_shuffled["label"]) & (df_shuffled["distance"] <= 0.95)
].reset_index(drop=True)

# df_error

In [ ]:
df_annotation_init = pd.read_csv(preprocessed_annotation_birds)

df_error_merged = df_error.merge(
    df_annotation_init[
        [
            col
            for col in df_annotation_init.columns
            if col not in df_error.columns or col in ["img_path", "couple_path"]
        ]
    ],
    on=["img_path", "couple_path"],
)

# df_error_merged

In [ ]:
image_utils.visualise_images_df(
    df=df_error_merged,
    n_images=1,
    file_type="jpg",
    crop_type=None,
    debug=True,
    transform_type="same",
)